### 利用闭包解决一个问题

问题:起始位置x=0,每次新赋值都会累加,并返回累加值

x = 0
3 -> result = 3

5 -> result = 8

9 -> result = 17

#### 非闭包解决

In [51]:
origin = 0

def go(step):
    new_pos = origin + step
    origin = new_pos
    return new_pos

print(go(3))
print(go(5))
print(go(9))

'''
    执行报错:
    UnboundLocalError: local variable 'origin' referenced before assignment
    赋值之前引用的局部变量“ origin”
    
    在Python中,变量的查找顺序可以解释这个报错.
        函数外的origin = 0,这行代码表示,origin为全局变量
        在go()中,origin = new_pos为局部变量
        new_pos = origin + step,这行表示要将局部变量origin的值参与运算后赋值给new_pos
        但是new_pos赋值操作却在定义局部变量origin之前,因此会报错
'''

UnboundLocalError: local variable 'origin' referenced before assignment

如何证明呢? 思考下这个问题.....


In [55]:
origin = 0

def go_new(step):
    new_pos = origin + step
    # origin = new_pos  注释掉这行代码就不会报错了,证明之前的判断是正确的
    return new_pos 

print(go_new(3))
print(go_new(5))
print(go_new(9))

3
5
9


纠正问题,让origin不是局部变量就可以了,加个global

不能使用nonlocal来声明,因为nonlocal声明的变量是***非全局作用域的变量***,***要求声明的变量已存在***,并不会创建新的变量

In [59]:
origin = 0

def go_new(step):
    global origin
    new_pos = origin + step
    origin = new_pos 
    return new_pos 

print(go_new(3))
print(go_new(5))
print(go_new(9))

3
8
17


#### 用闭包思想解决这个问题
原理:
在使用闭包的过程中,一旦外函数被调用一次返回了内函数的引用,虽然每次调用内函数,是开启一个函数执行过后消亡,但是闭包变量实际上只有一份,每次开启内函数都在使用同一份闭包变量

In [17]:
def go_outer():
    position = 0
    def go_inner(step):
        nonlocal position
        position += step
        return position
    return go_inner

test = go_outer()
print(test(3))
print(test(5))
print(test(9))

3
8
17


In [20]:
# 等价于以下代码:
position = 0
def go_outer():
    def go_inner(step):
        global position  # 不推荐,全局变量尽量少定义,推荐用nonlocal
        position += step
        return position
    return go_inner

test = go_outer()
print(test(3))
print("position:", position)
print(test(5))
print("position:", position)
print(test(9))
print("position:", position)

print("--------------分隔符--------------------")
# 不推荐原因:全局变量很容易在其他函数在也被使用,如果使用同名变量,会变量污染,得不到想要的结果
def go3():
    global position
    position +=1 
    return position
print(go3())  # 结果为18,在上一个函数作用的基础上+1


3
position: 3
8
position: 8
17
position: 17
--------------分隔符--------------------
18


In [22]:
# 等价于以下代码:
origin = 0
def go_outer(position):
    def go_inner(step):
        nonlocal position
        new_pos = position + step
        position = new_pos
        return new_pos
    return go_inner

test = go_outer(origin)
print(test(3))
print("origin:", origin)
print(test(5))
print("origin:", origin)
print(test(9))
print("origin:", origin)

3
origin: 0
8
origin: 0
17
origin: 0
